In [60]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [61]:
%%bash -s "$PLATFORM" -j 4
make PLATFORM=$1

rm -f -- image-demo-CWLITEARM.hex
rm -f -- image-demo-CWLITEARM.eep
rm -f -- image-demo-CWLITEARM.cof
rm -f -- image-demo-CWLITEARM.elf
rm -f -- image-demo-CWLITEARM.map
rm -f -- image-demo-CWLITEARM.sym
rm -f -- image-demo-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- image.s ../../../src/chiparmour.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- image.d ../../../src/chiparmour.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- image.i ../../../src/chiparmour.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc.exe (GNU Tools for ARM Embedded Processors 6-2017-q1-update) 6.3.1 20170215 (release) [ARM/embedded-6-branch revision 245512]
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: image.c
arm-none-eabi-gcc -c -mcpu=cortex-

In [3]:
%run "Helper_Scripts/Setup_Generic.ipynb"
scope.clock.adc_src = "clkgen_x1"

Serial baud rate = 38400


In [62]:
cw.program_target(scope, prog, "image-demo-CWLITEARM.hex")

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 7547 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 7547 bytes
Serial baud rate = 38400


In [63]:
def glitch_success(resp):
    return "CA Demo Image" in resp

In [65]:
target.flush()
scope.arm()
reset_target(scope)
scope.io.hs2 = "clkgen"
scope.clock.adc_src = "clkgen_x1"

ret = scope.capture()
if ret:
    print("Scope capture timed out")
response = target.read(timeout=10)
print(response)
print(scope.adc.trig_count)
print(glitch_success(response))

 RTOS Booted!


3186
False


In [66]:
#clock glitching
from collections import namedtuple
scope.io.glitch_hp=False
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"

scope.io.hs2 = "glitch"

Range = namedtuple('Range', ['min', 'max', 'step'])
width_range = Range(-8.98, -7, 1)
scope.glitch.offset_fine = 0
#scope.glitch.offset = -39.84
offset_range = Range(-38, -37, 0.4)


print(scope.glitch)

clk_src     = clkgen
width       = -8.984375
width_fine  = 0
offset      = -36.71875
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 864
repeat      = 1
output      = clock_xor



In [10]:
# voltage glitching
from collections import namedtuple
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.offset_fine = 24
Range = namedtuple('Range', ['min', 'max', 'step'])
width_range = Range(28.125, 29, 0.4)
#scope.glitch.offset = -39.84
offset_range = Range(-21, -18, 0.4)
scope.glitch.repeat=1
def glitch_on(scope):
    scope.io.glitch_lp = False
    scope.io.glitch_hp = True
def glitch_off(scope):
    scope.gio.glitch_hp = False
glitch_on(scope)

#offset_range = Range(-49, -30, 1)
#scope.glitch.ext_offset = 37
#scope.glitch.repeat = 10

print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 24
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = glitch_only



In [69]:
from collections import namedtuple
from tqdm import tnrange
f = open("log.txt", "w")

scope.glitch.width = width_range.min
scope.glitch.offset = offset_range.min
attack1_data = []
ext_offset_range = Range(500, 1500, 1)
sample_size = 5
scope.glitch.repeat = 1

while scope.glitch.offset < offset_range.max:
    while scope.glitch.width < width_range.max:
        for j in tnrange(ext_offset_range.min, ext_offset_range.max, ext_offset_range.step, leave=False):
            successes = 0
            scope.glitch.ext_offset = j
            for i in range(sample_size):
                scope.arm()
                reset_target(scope)
                ret = scope.capture()
                if ret:
                    print('Timeout happened during acquisition')

                response = target.read(timeout = 10)

                # for table display purposes
                success = glitch_success(response)
                if success:
                    print("Success")
                    print(response)
                    successes += 1
                if "RTOS" not in response:
                    print("Crashed")
                #f.write("___________________")
                #f.write(response)
                #f.write("-------------------")

            attack1_data.append([scope.glitch.width, scope.glitch.offset, successes/sample_size, scope.glitch.ext_offset, repr(response)])
            # run aux stuff that should happen after trace here
        scope.glitch.width += width_range.step
    scope.glitch.offset += offset_range.step
    scope.glitch.width = width_range.min
    print(scope.glitch)
print("Done glitching")

Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed
Crashed


KeyboardInterrupt: 

In [70]:
for row in attack1_data:
    if row[2] > 0:
        print(row)

In [9]:
f.close()

In [ ]:
#859????